# 0.0 Imports

In [ ]:
import pickle
import pandas   as pd

from sklearn import preprocessing as pp

import funcoes_suporte as fs

## 0.1 Help functions

In [3]:
fs.jupyter_settings(altura= 18, largura= 14, fonte= 11)
fs.supressao_notacao()

## 0.2 Load interim data

In [ ]:
x_treino = pickle.load( open( "../data/interim/x_treino.pkl", "rb" ) )
y_treino = pickle.load( open("../data/interim/y_treino.pkl", "rb"))
df = x_treino.copy()
df['resposta'] = y_treino.values
df.head()

,id,genero,idade,codigo_regiao,contato_cliente,cnh,idade_veiculo,veiculo_danificado,seguro_previo_automovel,premio_anual,cliente_dias_contrato,resposta
55914,55915,Male,33,29.000,124.000,1,1-2 Year,Yes,0,26761.000,139,0
179876,179877,Male,55,28.000,26.000,1,> 2 Years,Yes,0,27875.000,285,1
300095,300096,Female,26,28.000,152.000,1,< 1 Year,No,1,43562.000,118,0
60078,60079,Male,57,38.000,124.000,1,1-2 Year,No,1,51365.000,249,0
312426,312427,Male,27,42.000,152.000,1,< 1 Year,Yes,0,26590.000,161,0


In [ ]:
del x_treino, y_treino

# 1.0 Data Preparation

In [7]:
df.columns

Index(['id', 'genero', 'idade', 'codigo_regiao', 'contato_cliente', 'cnh',
       'idade_veiculo', 'veiculo_danificado', 'seguro_previo_automovel',
       'premio_anual', 'cliente_dias_contrato', 'resposta'],
      dtype='object')

## 1.1 Normalização

In [ ]:
ss = pp.StandardScaler()

# premio_anual
df['premio_anual'] = ss.fit_transform( df[['premio_anual']].values)

In [6]:
pickle.dump(ss,open('../src/features/premio_anual_normalize.pkl','wb'))

## 1.2 Reescala

In [ ]:
mms_idade = pp.MinMaxScaler()
mms_cdc = pp.MinMaxScaler()

# idade
df[['idade']] = mms_idade.fit_transform( df[['idade']].values )

# cliente_dias_contrato

df[['cliente_dias_contrato']] = mms_cdc.fit_transform( df[['cliente_dias_contrato']].values )

In [8]:
pickle.dump( mms_idade, open('../src/features/mms_idade.pkl ','wb' ) )
pickle.dump( mms_cdc, open('../src/features/mms_cdc.pkl','wb' ) )

## 1.3 Encoder

In [ ]:
# cnh - tem ou não algo 0 1 - já está em LabelEncoder
# seguro_previo_automovel - já está em LabelEncoder
# veiculo_danificado - já está em LabelEncoder

# genero - OneHotEncoder / TargetEncoder
target_encode_genero = df.groupby(['genero'])['resposta'].mean()
df.loc[:, 'genero'] = df['genero'].map(target_encode_genero)

# codigo_regiao -  One Hot Encoding / Frequency Encoding / Target Encoding / Weighted Target Encoding
target_encode_codigo_regiao = df.groupby(['codigo_regiao'])['resposta'].mean()
df.loc[:, 'codigo_regiao'] = df['codigo_regiao'].map(target_encode_codigo_regiao)

# idade_veiculo - One Hot Encoding / Order Encoding / Frequency Encoding
df = pd.get_dummies(df, prefix=['idade_veiculo'], columns=['idade_veiculo'])

# contato_cliente - Target Encoding / Frequency Encoding
fe_contato_cliente = df.groupby(['contato_cliente']).size() / len(df)
df.loc[:, 'contato_cliente'] = df['contato_cliente'].map(fe_contato_cliente)

In [ ]:
pickle.dump( target_encode_genero, open('../src/features/target_encode_genero.pkl', 'wb' ) )
pickle.dump( target_encode_codigo_regiao, open('../src/features/target_encode_codigo_regiao.pkl', 'wb' ) )
pickle.dump( fe_contato_cliente , open('../src/features/fe_contato_cliente.pkl', 'wb' ) )

# 1.4 Transofrmação

# 2.0 Export Pickle interim Data

In [ ]:
# x_treino
x_treino = df.drop(columns = ['resposta'],  axis=1 ).reset_index(drop=True)
pickle.dump(x_treino,open('../data/interim/x_treino.pkl','wb'))

# y_treino
y_treino = df['resposta'].copy()
pickle.dump(y_treino,open('../data/interim/y_treino.pkl','wb'))

In [ ]:
del df, x_treino, y_treino